In [17]:
import math
import numpy as np

In [19]:
def find_entropy(l):
    H = np.array([-i*np.log2(i) for i in l]).sum()
    return H

print(find_entropy([0.5, 0.5]))
print(find_entropy([0.9, 0.1]))

1.0
0.4689955935892812


In [47]:
import pandas as pd

df = pd.read_csv('Datasets/tennis.txt', delimiter="\t", header=None, names=['Outlook', 'Temp', 'Humidity', 'Wind', 'Decision'])
df

,Outlook,Temp,Humidity,Wind,Decision
1,Sunny,Hot,High,Weak,No
2,Sunny,Hot,High,Strong,No
3,Overcast,Hot,High,Weak,Yes
4,Rain,Mild,High,Weak,Yes
5,Rain,Cool,Normal,Weak,Yes
6,Rain,Cool,Normal,Strong,No
7,Overcast,Cool,Normal,Strong,Yes
8,Sunny,Mild,High,Weak,No
9,Sunny,Cool,Normal,Weak,Yes
10,Rain,Mild,Normal,Weak,Yes


In [105]:
def conditional_prob(df, c1, c2, condition):
    df_new = df[df[c1] == condition][c2]
    s = df_new.unique()
    population_size = len(df_new)
    pr = {}
    for i in s:
        pr[i] = len(df[(df[c1] == condition) & (df[c2]== i)]) / population_size
    
    return pr

# # what are the probabilities of Play  given Wind is Weak?
# print(conditional_prob(df,'Wind', 'Play', 'Weak'))

# # what are the probabilities of Play given Wind is Strong?
# print(conditional_prob(df, 'Wind', 'Play', 'Strong'))

In [66]:
df['Decision'].value_counts()

Yes    9
No     5
Name: Decision, dtype: int64

In [50]:
print(find_entropy([(9/14), (5/14)]))

0.9402859586706311


In [58]:
df[df['Wind'] == "Weak"]['Decision'].value_counts()

Yes    6
No     2
Name: Decision, dtype: int64

In [52]:
print(find_entropy([(6/8), (2/8)]))

0.8112781244591328


In [54]:
df[df['Wind'] == "Strong"]['Decision'].value_counts()

No     3
Yes    3
Name: Decision, dtype: int64

In [55]:
print(find_entropy([(3/6), (3/6)]))

1.0


In [59]:
df['Wind'].value_counts()

Weak      8
Strong    6
Name: Wind, dtype: int64

In [61]:
print("Weak Wind: " + str(8/14))
print("Strong Wind: " + str(6/14))

Weak Wind: 0.5714285714285714
Strong Wind: 0.42857142857142855


In [64]:
0.9402859586706311 - (0.5714285714285714 * 0.8112781244591328) - (0.42857142857142855 * 1)

0.048127030408269544

In [120]:
def info_gain(df, feature, decision):
    l1 = [df[decision].value_counts()[i]/sum(df[decision].value_counts()) for i in range(len(df[decision].value_counts()))]
    des_entropy = find_entropy(l1)
    
#     feature_H = [df[feature].value_counts()[i]/sum(df[feature].value_counts()) for i in range(len(df[feature].value_counts()))]
    
    features = dict(df[feature].value_counts())
    prob_feature = {}
    for key, value in features.items():
        prob_feature[key] = value/sum(features.values())
        
    conditions = df[feature].unique()
    cond = {}
    for condition in conditions:
        cond[condition] = conditional_prob(df, feature, decision, condition)

    temp = 0
    for key, value in cond.items():
        prob_condition = list(cond[key].values())
        temp += prob_feature[key]*find_entropy(prob_condition)

    info_g = des_entropy - temp
    
    return info_g

print("Wind: " + str(info_gain(df, 'Wind', 'Decision')))
print("Outlook: " + str(info_gain(df, 'Outlook', 'Decision')))
print("Temp: " + str(info_gain(df, 'Temp', 'Decision')))
print("Humidity: " + str(info_gain(df, 'Humidity', 'Decision')))
print("Decision: " + str(info_gain(df, 'Decision', 'Decision')))

Wind: 0.04812703040826949
Outlook: 0.24674981977443933
Temp: 0.02922256565895487
Humidity: 0.15183550136234159
Decision: 0.9402859586706311


In [123]:
info_gain(df[df['Outlook'] == 'Sunny'], 'Temp', 'Decision')

0.5709505944546686

In [124]:
print("Wind: " + str(info_gain(df[df['Outlook'] == 'Sunny'], 'Wind', 'Decision')))
print("Temp: " + str(info_gain(df[df['Outlook'] == 'Sunny'], 'Temp', 'Decision')))
print("Humidity: " + str(info_gain(df[df['Outlook'] == 'Sunny'], 'Humidity', 'Decision')))

Wind: 0.01997309402197489
Temp: 0.5709505944546686
Humidity: 0.9709505944546686


In [127]:
print("Wind: " + str(info_gain(df[df['Outlook'] == 'Rain'], 'Wind', 'Decision')))
print("Temp: " + str(info_gain(df[df['Outlook'] == 'Rain'], 'Temp', 'Decision')))
print("Humidity: " + str(info_gain(df[df['Outlook'] == 'Rain'], 'Humidity', 'Decision')))

Wind: 0.9709505944546686
Temp: 0.01997309402197489
Humidity: 0.01997309402197489


In [132]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.tree import export_graphviz
# import pydotplus

# read in the tennis data, need the extra parameters since it's a txt file
data = pd.read_csv('Datasets/tennis.txt', delimiter="\t", header=None, names=['Outlook', 'Temp', 'Humidity', 'Wind', 'Play'])
print(data)

# encode the data so we can use it with our decision tree,
# by converting categories into numbers
data_encoded = data.apply(preprocessing.LabelEncoder().fit_transform)
print(data_encoded)

# create our decision tree classifier with entropy
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3)

# one_hot_data = pd.get_dummies(data[['a', 'b', 'c', 'd']], drop_first=True)
# print(one_hot_data)

# provide our feature array and target array (1-item),
# and train the model using a decision tree
clf.fit(data_encoded[['Outlook', 'Temp', 'Humidity', 'Wind']], data_encoded['Play'])

# export our decision tree into data that can be plotted
dot_data = export_graphviz(clf, out_file=None, feature_names=['Outlook', 'Temp.', 'Humidity', 'Wind'])

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_png('tennis_tree.png')

     Outlook  Temp Humidity    Wind Play
1      Sunny   Hot     High    Weak   No
2      Sunny   Hot     High  Strong   No
3   Overcast   Hot     High    Weak  Yes
4       Rain  Mild     High    Weak  Yes
5       Rain  Cool   Normal    Weak  Yes
6       Rain  Cool   Normal  Strong   No
7   Overcast  Cool   Normal  Strong  Yes
8      Sunny  Mild     High    Weak   No
9      Sunny  Cool   Normal    Weak  Yes
10      Rain  Mild   Normal    Weak  Yes
11     Sunny  Mild   Normal  Strong  Yes
12  Overcast  Mild     High  Strong  Yes
13  Overcast   Hot   Normal    Weak  Yes
14      Rain  Mild     High  Strong   No
    Outlook  Temp  Humidity  Wind  Play
1         2     1         0     1     0
2         2     1         0     0     0
3         0     1         0     1     1
4         1     2         0     1     1
5         1     0         1     1     1
6         1     0         1     0     0
7         0     0         1     0     1
8         2     2         0     1     0
9         2     0        

NameError: name 'pydotplus' is not defined